In [1]:
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
from modeler.modeler import Modeler as m

In [49]:
data = pd.read_csv("Twitch_game_data.csv")

In [50]:
data["date"] = [datetime(int(row[1]["Year"]),int(row[1]["Month"]),1) for row in data.iterrows()]
data["Hours_watched"] = [int(x) for x in data["Hours_watched"]]
data["Hours_Streamed"] = [int(x.split(" ")[0]) for x in data["Hours_Streamed"]]

In [51]:
data["efficiency"] = data["Hours_watched"] / data["Hours_Streamed"]

In [76]:
gamz = []
metric = "Avg_viewers"
for game in data["Game"].unique():
    game_data = data[data["Game"]==game]
    for i in range(3):
        game_data[str(i)] = game_data[metric].shift(1)
    game_data["y"] = game_data[metric].shift(-2)
    gamz.append(game_data)

<ipython-input-76-43d9a7b0bd50>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  game_data[str(i)] = game_data[metric].shift(1)
<ipython-input-76-43d9a7b0bd50>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  game_data["y"] = game_data[metric].shift(-2)


In [77]:
model_data = pd.concat(gamz).dropna()
prediction_data = pd.concat(gamz)

In [78]:
model_data.columns

Index(['Rank', 'Game', 'Month', 'Year', 'Hours_watched', 'Hours_Streamed',
       'Peak_viewers', 'Peak_channels', 'Streamers', 'Avg_viewers',
       'Avg_channels', 'Avg_viewer_ratio', 'date', 'efficiency', '0', '1', '2',
       'y'],
      dtype='object')

In [79]:
model_data = model_data.sample(frac=1)

In [80]:
training_data = model_data.iloc[:int(model_data.index.size * 0.7)]
testing_data = model_data.iloc[int(model_data.index.size * 0.7):]
factors = ["Month","Hours_watched","Streamers","Avg_viewers","Avg_channels","0","1","2"]

In [81]:
X = training_data[factors]
y = training_data[["y"]]
sk_model = m.sk_regression({"X":X,"y":y})
xgb_model = m.xgb_regression({"X":X,"y":y})

c:\users\eric\appdata\local\programs\python\python38\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
c:\users\eric\appdata\local\programs\python\python38\lib\site-packages\sklearn\model_selection\_validation.py:683: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\users\eric\appdata\local\programs\python\python38\lib\site-packages\sklearn\model_selection\_validation.py", line 674, in _score
    scores = scorer(estimator, X_test, y_test)
  File "c:\users\eric\appdata\local\programs\python\python38\lib\site-packages\sklearn\metrics\_scorer.py", line 199, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true,
  File "c:\users\eric\appdata\local\programs\python\p

c:\users\eric\appdata\local\programs\python\python38\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the test scores are non-finite: [0.78459968        nan 0.78459968]
  warnings.warn(


In [82]:
sk_model.append(xgb_model)

In [83]:
model = pd.DataFrame(sk_model).sort_values(by="score",ascending=False).iloc[0]

In [84]:
predictions = model["model"].predict(testing_data[factors])

In [85]:
testing_data["predictions"] = [x[0] for x in predictions]
testing_data.sort_values("date",ascending=True,inplace=True)

<ipython-input-85-0c61b2cb4cc8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  testing_data["predictions"] = [x[0] for x in predictions]
<ipython-input-85-0c61b2cb4cc8>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  testing_data.sort_values("date",ascending=True,inplace=True)


In [86]:
upcoming_data = prediction_data[prediction_data["date"] >= datetime(2021,6,1)]

In [87]:
upcoming_data.drop("y",axis=1,inplace=True)

c:\users\eric\appdata\local\programs\python\python38\lib\site-packages\pandas\core\frame.py:3990: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [88]:
upcoming_data.dropna(inplace=True)

<ipython-input-88-ca08578af31d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  upcoming_data.dropna(inplace=True)


In [89]:
upcoming_predictions = model["model"].predict(upcoming_data[factors])
upcoming_data["predictions"] = [x[0] for x in upcoming_predictions]
upcoming_data["score"] = model["score"]

<ipython-input-89-373f451345ca>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  upcoming_data["predictions"] = [x[0] for x in upcoming_predictions]
<ipython-input-89-373f451345ca>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  upcoming_data["score"] = model["score"]


In [90]:
upcoming_data["gain"] = (upcoming_data["predictions"] - upcoming_data[metric]) / upcoming_data[metric]

<ipython-input-90-0d215fa461ca>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  upcoming_data["gain"] = (upcoming_data["predictions"] - upcoming_data[metric]) / upcoming_data[metric]


In [91]:
upcoming_data.sort_values("gain",ascending=False).head(20)

,Rank,Game,Month,Year,Hours_watched,Hours_Streamed,Peak_viewers,Peak_channels,Streamers,Avg_viewers,Avg_channels,Avg_viewer_ratio,date,efficiency,0,1,2,predictions,score,gain
13046,47,Resident Evil Village,6,2021,4694663,234473,50271,1072,32479,6529,326,20.02,2021-06-01,20.022190,67854.0,67854.0,67854.0,16093.381380,0.888588,1.464908
13193,194,Biomutant,6,2021,719024,102352,19200,719,12208,1000,142,7.03,2021-06-01,7.025012,7442.0,7442.0,7442.0,2381.580739,0.888588,1.381581
13120,121,Subnautica: Below Zero,6,2021,1403450,73478,14327,360,8477,1951,102,19.10,2021-06-01,19.100275,14539.0,14539.0,14539.0,4128.116462,0.888588,1.115898
13185,186,Resident Evil 7 biohazard,6,2021,764140,84771,31608,299,15370,1062,117,9.01,2021-06-01,9.014168,4643.0,4643.0,4643.0,2111.436444,0.888588,0.988170
13194,195,Overcooked! 2,6,2021,715442,79740,31697,823,29331,995,110,8.97,2021-06-01,8.972185,433.0,433.0,433.0,1774.472439,0.888588,0.783389
13199,200,Tabletop RPGs,6,2021,688130,23602,20018,122,2567,957,32,29.16,2021-06-01,29.155580,3770.0,3770.0,3770.0,1642.464655,0.888588,0.716264
13153,154,Monster Hunter Rise,6,2021,913315,74045,26630,275,7768,1270,102,12.33,2021-06-01,12.334594,3486.0,3486.0,3486.0,1949.912731,0.888588,0.535364
13084,85,Knockout City,6,2021,2194194,179088,99690,1144,33414,3051,249,12.25,2021-06-01,12.252044,8496.0,8496.0,8496.0,4680.170860,0.888588,0.533979
13088,89,Roblox,6,2021,2103682,372196,20358,885,58979,2925,517,5.65,2021-06-01,5.652081,4006.0,4006.0,4006.0,4395.432463,0.888588,0.502712
13198,199,Little Nightmares II,6,2021,695130,27581,43518,105,6128,966,38,25.20,2021-06-01,25.203220,1786.0,1786.0,1786.0,1441.187086,0.888588,0.491912


In [92]:
game = "Genshin Impact"
upcoming_data[upcoming_data["Game"]==game]

,Rank,Game,Month,Year,Hours_watched,Hours_Streamed,Peak_viewers,Peak_channels,Streamers,Avg_viewers,Avg_channels,Avg_viewer_ratio,date,efficiency,0,1,2,predictions,score,gain
13025,26,Genshin Impact,6,2021,10688912,580456,53282,2460,58266,14866,807,18.41,2021-06-01,18.414681,17198.0,17198.0,17198.0,16092.047561,0.888588,0.082473
